# Ridge and Lasso Regression - Lab

## Introduction

In this lab, you'll practice your knowledge on Ridge and Lasso regression!

## Objectives

You will be able to:

- Use Lasso and ridge regression in Python
- Compare Lasso and Ridge with standard regression

## Housing Prices Data

Let's look at yet another house pricing data set.

In [41]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Housing_Prices/train.csv')

Look at df.info

In [42]:
# Your code here
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

First, make a selection of the data by removing some of the data with `dtype = object`, this way our first model only contains **continuous features**

Make sure to remove the SalesPrice column from the predictors (which you store in `X`), then replace missing inputs by the median per feature.

Store the target in `y`.

In [43]:
# Load necessary packages
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# remove "object"-type features and SalesPrice from `X`
features = [col for col in df.columns if df[col].dtype in [np.float64, np.int64] and col!='SalePrice']
X = df[features]

# Impute null values
for col in X:
    med = X[col].median()
    X[col].fillna(value = med, inplace = True)

# Create y
y = df.SalePrice


Look at the information of `X` again

In [44]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 37 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtFinSF2       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
LowQualFinSF     1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
BsmtHalfBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
F

## Let's use this data to perform a first naive linear regression model

Compute the R squared and the MSE for both train and test set.

In [45]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error

# Split in train and test
X_train, X_test, y_train, y_test = train_test_split(X,y)
# Fit the model and print R2 and MSE for train and test
linreg = LinearRegression()
linreg.fit(X_train, y_train)

print('Train Rsq:   ', linreg.score(X_train, y_train))
print('Test  Rsq:   ', linreg.score(X_test,y_test))
print('Train MSE:   ', mean_squared_error(y_train, linreg.predict(X_train)))
print('Test  MSE:   ',mean_squared_error(y_test, linreg.predict(X_test)))

Train Rsq:    0.8048861761646349
Test  Rsq:    0.8186866152041727
Train MSE:    1213142634.794117
Test  MSE:    1188738509.417328


## Normalize your data

We haven't normalized our data, let's create a new model that uses `preprocessing.scale` to scale our predictors!

In [46]:
from sklearn import preprocessing

# Scale the data and perform train test split
Xscaled = preprocessing.scale(X)

X_train, X_test, y_train, y_test = train_test_split(Xscaled, y)


Perform the same linear regression on this data and print out R-squared and MSE.

In [47]:
# Your code here
linreg = LinearRegression()
linreg.fit(X_train, y_train)

print('Train Rsq:   ', linreg.score(X_train, y_train))
print('Test  Rsq:   ', linreg.score(X_test,y_test))
print('Train MSE:   ', mean_squared_error(y_train, linreg.predict(X_train)))
print('Test  MSE:   ',mean_squared_error(y_test, linreg.predict(X_test)))

Train Rsq:    0.7995964984889294
Test  Rsq:    0.8392538791145157
Train MSE:    1260340709.5029516
Test  MSE:    1022349575.0855967


## Include dummy variables

Your model hasn't included dummy variables so far: let's use the "object" variables again and create dummies

In [48]:
# Create X_cat which contains only the categorical variables
cat_features = [col for col in df.columns if df[col].dtype in [np.object]]
X_cat = df[cat_features]
np.shape(X_cat)

(1460, 43)

In [49]:
# Make dummies
X_cat = pd.get_dummies(X_cat)
np.shape(X_cat)

(1460, 252)

Merge `x_cat` together with our scaled `X` so you have one big predictor dataframe.

In [50]:
# Your code here
X_all = pd.concat([pd.DataFrame(Xscaled), X_cat], axis=1)

Perform the same linear regression on this data and print out R-squared and MSE.

In [51]:
# Your code here
X_train, X_test, y_train, y_test = train_test_split(X_all, y)

linreg = LinearRegression()
linreg.fit(X_train, y_train)

print('Train Rsq:   ', linreg.score(X_train, y_train))
print('Test  Rsq:   ', linreg.score(X_test,y_test))
print('Train MSE:   ', mean_squared_error(y_train, linreg.predict(X_train)))
print('Test  MSE:   ',mean_squared_error(y_test, linreg.predict(X_test)))


Train Rsq:    0.803419411037808
Test  Rsq:    -3.0453485662950725e+22
Train MSE:    1257134738.3945205
Test  MSE:    1.8389806324036263e+32


Notice the severe overfitting above; our training R squared is quite high, but the testing R squared is negative! Our predictions are far off. Similarly, the scale of the Testing MSE is orders of magnitude higher than that of the training.

## Perform Ridge and Lasso regression

Use all the data (normalized features and dummy categorical variables) and perform Lasso and Ridge regression for both! Each time, look at R-squared and MSE.

## Lasso

With default parameter (alpha = 1)

In [52]:
# Your code here
from sklearn.linear_model import Lasso, Ridge
lasso = Lasso(alpha=1)
lasso.fit(X_train, y_train)

print('Train Rsq:   ', lasso.score(X_train, y_train))
print('Test  Rsq:   ', lasso.score(X_test,y_test))
print('Train MSE:   ', mean_squared_error(y_train, lasso.predict(X_train)))
print('Test  MSE:   ',mean_squared_error(y_test, lasso.predict(X_test)))

Train Rsq:    0.9326525591817914
Test  Rsq:    0.7506723461773402
Train MSE:    430687525.3630594
Test  MSE:    1505603435.932204


With a higher regularization parameter (alpha = 10)

In [53]:
# Your code here
lasso = Lasso(alpha=10)
lasso.fit(X_train, y_train)

print('Train Rsq:   ', lasso.score(X_train, y_train))
print('Test  Rsq:   ', lasso.score(X_test,y_test))
print('Train MSE:   ', mean_squared_error(y_train, lasso.predict(X_train)))
print('Test  MSE:   ',mean_squared_error(y_test, lasso.predict(X_test)))

Train Rsq:    0.9307018836188934
Test  Rsq:    0.7101033544121149
Train MSE:    443162114.16352266
Test  MSE:    1750585540.6347725


## Ridge

With default parameter (alpha = 1)

In [54]:
# Your code here
ridge = Ridge()
ridge.fit(X_train, y_train)
lasso = Lasso()
lasso.fit(X_train, y_train)

print('Train Rsq:   ', ridge.score(X_train, y_train))
print('Test  Rsq:   ', ridge.score(X_test,y_test))
print('Train MSE:   ', mean_squared_error(y_train, ridge.predict(X_train)))
print('Test  MSE:   ',mean_squared_error(y_test, ridge.predict(X_test)))

Train Rsq:    0.92876169886997
Test  Rsq:    0.7043481682453856
Train MSE:    455569614.10871863
Test  MSE:    1785339118.6442957


With default parameter (alpha = 10)

In [55]:
# Your code here
ridge = Ridge(alpha=10)
ridge.fit(X_train, y_train)
lasso = Lasso()
lasso.fit(X_train, y_train)

print('Train Rsq:   ', ridge.score(X_train, y_train))
print('Test  Rsq:   ', ridge.score(X_test,y_test))
print('Train MSE:   ', mean_squared_error(y_train, ridge.predict(X_train)))
print('Test  MSE:   ',mean_squared_error(y_test, ridge.predict(X_test)))

Train Rsq:    0.9152820654738436
Test  Rsq:    0.718343914424697
Train MSE:    541771998.0396796
Test  MSE:    1700823649.8909009


## Look at the metrics, what are your main conclusions?   

Conclusions here

## Compare number of parameter estimates that are (very close to) 0 for Ridge and Lasso

Compare with the total length of the parameter space and draw conclusions!

In [63]:
# number of Ridge params almost zero
print(sum(abs(ridge.coef_) < 10**(-5)))
print(len(ridge.coef_))

6
289


In [64]:
# number of Lasso params almost zero
print(sum(abs(lasso.coef_) < 10**(-5)))
print(len(lasso.coef_))

35
289


Lasso was very effective to essentially perform variable selection and remove about 25% of the variables from your model!

## Summary

Great! You now know how to perform Lasso and Ridge regression.